# Silver layer

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql.functions import col, month, year

from pyspark.sql.functions import concat, col, lit

### Data accessing

#### use your credentials here

### Data loading

In [0]:
df_cal = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cus = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_procat = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_pro = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
 df_ret = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Returns')

In [0]:
 df_Sales = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_ter = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_subcat = spark.read.format('csv').option('inferschema',True).option('header',True).load('abfss://bronze@awstoragedatalake123456.dfs.core.windows.net/Product_Subcategories')

## Transformations

##### transformations on calender file

In [0]:
df_cal = df_cal.withColumn('Month', month(col('Date')))

In [0]:
df_cal = df_cal.withColumn('Year',year(col('Date')))


In [0]:
df_cal.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

##### transformation on customers table

In [0]:
df_cus = df_cus.withColumn("FullName",concat_ws(' ',col('Prefix'),col('FirstName'),col('LastName')))

In [0]:
df_cus.write.format('parquet')\
                  .mode('append')\
                  .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Customers")\
                  .save()

##### Product categories

In [0]:
df_procat.write.format('parquet')\
        .mode('append')\
        .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Product_Categories")\
        .save()

##### products

In [0]:
df_pro = df_pro.withColumn('ProductSKU',split(col('ProductSKU'), '-')[0])\
       .withColumn('ProductName',split(col('ProductName'), ' ')[0])

In [0]:
df_pro.write.format('parquet')\
        .mode('append')\
        .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Products")\
        .save()

##### returns

In [0]:
df_ret.write.format('parquet')\
        .mode('append')\
        .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Returns")\
        .save()

#####Territories

In [0]:
df_ter.write.format('parquet')\
        .mode('append')\
        .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Territories")\
        .save()

#### sales data

In [0]:
df_Sales = df_Sales.withColumn('StockDate',to_timestamp(col('StockDate')))

In [0]:
df_Sales = df_Sales.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_Sales = df_Sales.withColumn('Multiply',col('OrderLineItem')*col('OrderQuantity'))

## Sales Analysis

In [0]:
df_Sales = df_Sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order'))

In [0]:
df_Sales.display()

OrderDate,total_order
2017-01-06,151
2017-01-27,142
2017-02-26,119
2017-01-24,173
2017-06-29,172
2017-02-16,124
2017-04-09,140
2017-02-28,162
2017-03-28,149
2017-06-30,136


Databricks visualization. Run in Databricks to view.

In [0]:
df_Sales.write.format('Parquet')\
              .mode('append')\
              .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Sales")\
              .save()

##### Subcategories data file 

In [0]:
df_subcat.write.format('Parquet')\
              .mode('append')\
              .option('path',"abfss://silver@awstoragedatalake123456.dfs.core.windows.net/AdventureWorks_Subcategories")\
              .save()